In [1]:
import pandas as pd
import numpy as np
import torch
import tqdm
from deeppavlov import build_model

In [3]:
model = build_model('rusentiment_convers_bert', download=True)

2023-04-12 22:48:16.592 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/classifiers/rusentiment_convers_bert/rusentiment_convers_bert_torch.tar.gz download because of matching hashes
c:\Users\Димас\venvs\main_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertF

In [13]:
data_sentiment = pd.read_csv("sentiment.csv")

In [109]:
age = 61
gender = 1

In [112]:
dataset_path = f'datasets/{age}_{gender}_0_150.csv'
try:
    df = pd.read_csv(dataset_path, delimiter='\t')
except:
    dataset_path = f'datasets/{age}_{gender}_0_150'
    df = pd.read_csv(dataset_path, delimiter='\t')
df.replace('þ<br />þ', '\n ', regex=True, inplace=True)
df.head()

,text,gender,age,numwords,userid,year
0,Мало кто может спеть «Кукушку» Цоя так же хоро...,1,61,17,101049741,2015
1,Автобус в Таллин: контактная информация,1,61,5,10360086,2014
2,В Петербурге живут самые читающие коты,1,61,6,10360086,2015
3,"Иринка родная, С Днем Рождения!!!))) Здорлвья ...",1,61,10,10360340,2021
4,"Коля, с Рождеством Христовым!!!",1,61,4,103606625,2021


In [113]:
data = df['text'].tolist()

In [114]:
batch_size = 500
batched_sentiment = []
for i in range(0, len(data), batch_size):
  batched_sentiment.append(model(data[i : i+batch_size]))

In [115]:
plus = 0
minus = 0
neutral = 0
speech = 0
for batch in batched_sentiment:
  for sent in batch:
    if sent == 'positive':
      plus += 1
    elif sent == 'negative':
      minus += 1
    elif sent == 'neutral':
      neutral += 1
    elif sent == 'speech':
      speech += 1

In [2]:
data_sentiment = pd.read_csv('sentiment.csv')
data_sentiment

,age,gender,positive,negative,neutral,speech
0,16,1,265630,137348,391052,89173
1,16,2,162755,129985,532093,56005
2,17,1,272967,138353,378102,98666
3,17,2,175376,138872,495780,66262
4,18,1,278213,133324,383801,101186
5,18,2,181456,140761,485964,70251
6,19,1,283494,123207,397761,102808
7,19,2,180638,137142,494811,73081
8,20,1,287515,110922,417101,102661
9,20,2,180244,130807,505819,75518


In [117]:
df_temp = pd.DataFrame({'age': [age], 'gender': [gender], 'positive': [plus], 'negative': [minus], 'neutral': [neutral], 'speech': [speech]})
df_temp

,age,gender,positive,negative,neutral,speech
0,61,1,56460,23419,243640,36624


In [118]:
data_sentiment = pd.concat((data_sentiment, df_temp))
data_sentiment

,age,gender,positive,negative,neutral,speech
0,63,1,46294,21290,189987,30357
1,62,1,50142,22596,218170,33182
0,61,1,56460,23419,243640,36624


In [119]:
data_sentiment.to_csv('sentiment.csv', index=False)